In [ ]:
import pandas as pd

def brandEquityScore_weekly(file_name, output_filename):
    
    #Output dataframe
    df_score = pd.DataFrame()
    
    #Metrics label
    xls = pd.ExcelFile(file_name, on_demand = True)
    equity_metrics = xls.sheet_names[1:]
    
    
    #Gathering sector and brand labels
    label_row = pd.read_excel(file_name, sheet_name=2)
    sector_rows = label_row.iloc[0,:]
    brand_rows = label_row.iloc[4,:]

    sectors = []
    for i in sector_rows:
        if i != 0:
            sectors.append(i)
    sectors = sectors[1:]

    brands = []
    for j in brand_rows:
        if j != 0:
            brands.append(j)
    brands = brands[1:]
    
    #brand and sector to a list
    brand_sector = []
    for i, j in zip(brands, sectors):
        brand_sector.append(i+"_"+j)
     
    df_score_melted = pd.DataFrame()
    #looping through each tab
    for i, n in enumerate(equity_metrics):
        df = pd.read_excel(file_name, skiprows=6 ,sheet_name=i+1, header=[0], index_col='Unnamed: 0')
    
    #Gather all column names containing either 'Score' or 'Attention'
        col = []
        for k in df.columns:
            if 'Score' in k:
                col.append(k)
            elif 'Attention' in k:
                col.append(k)
                
        df_score = df.loc[:,col]
        df_score.columns = brand_sector
        
        df_score.index.rename('Week end date')       
        df_score = df_score[df_score.index.dayofweek == 6] 
        
        df_score.reset_index(inplace=True)
        df_score.rename(columns={'index':'date'},inplace=True)
        
        df_tab_melt = pd.melt(df_score, id_vars=['date'], value_vars=brand_sector, var_name='brand_sector', value_name='score')
        
        df_tab_melt['brand'] = df_tab_melt['brand_sector'].apply(lambda x: x.split('_')[0])
        df_tab_melt['sector'] = df_tab_melt['brand_sector'].apply(lambda x: x.split('_')[1])

        df_tab_melt['metric'] = xls.sheet_names[i+1]
        df_tab_melt.drop(columns=['brand_sector'], axis=1, inplace=True)
        
        df_score_melted = df_score_melted.append(df_tab_melt)
    
    df_score_melted.to_csv(output_filename, encoding='utf-8', index = False)